# VCN Peering

In this exercise we will be using OpenTofu (an open-source fork of Terraform) and Oracle Cloud Infrastructure's Free Tier to peer VCNs.

[OpenTofu](https://opentofu.org)

[Oracle Cloud Free Tier](https://www.oracle.com/au/cloud/free/)

## Authentication

When beginning a session you will need to authenticate:

```bash
oci session authenticate
```
A session lasts one hour. Don't forget to refresh your session to avoid having to authenticate again.


In [ ]:
!oci session refresh --profile harleycalvert

# Set Up OCI Terraform

[Set Up OCI Terraform](https://docs.oracle.com/en-us/iaas/developer-tutorials/tutorials/tf-provider/01-summary.htm)

## Install OpenTofu

In [ ]:
!snap install --classic opentofu

In [ ]:
!tofu -v

## Create RSA Keys

In [ ]:
!mkdir $HOME/.oci

In [ ]:
!openssl genrsa -out $HOME/.oci/oci_rsa_key.pem 2048

In [ ]:
!chmod 600 $HOME/.oci/oci_rsa_key.pem

In [ ]:
!openssl rsa -pubout -in $HOME/.oci/oci_rsa_key.pem -out $HOME/.oci/oci_rsa_key.pem.pub

In [ ]:
!(cd $HOME/.oci/ && ls)

In [ ]:
!cat $HOME/.oci/oci_rsa_key.pem.pub

Add the public key to your user account.

In the OCI Console's top navigation bar, click the Profile menu, and then go to My profile.
- Click API Keys.
- Click Add API Key.
- Select Paste Public Keys.
- Paste value from previous step, including the lines with BEGIN PUBLIC KEY and END PUBLIC KEY.
- Click Add.
- Copy the Configuration file preview.
- Paste the Configuration file preview into ~/.oci/config

You have now set up the RSA keys to connect to your OCI account.

## Add API Key-Based Authentication

In [ ]:
!mkdir tf-provider

## Gather Required Information
Collect the following credential information from the OCI Console.

- Tenancy OCID
- User OCID
- Fingerprint
- Region 
  - ap-melbourne-1

Collect the following information from your environment.

- Private Key Path 
  - /home/harley/.oci/oci_rsa_key.pem
  
Put the information in provider.tf like so:

In [ ]:
%%writefile ./tf-provider/provider.tf


provider "oci" {  
  tenancy_ocid = "<tenancy-ocid>"
  user_ocid = "<user-ocid>" 
  private_key_path = "<rsa-private-key-path>"
  fingerprint = "<fingerprint>"
  region = "<region-identifier>"
}

In [ ]:
!code ./tf-provider/provider.tf

In [ ]:
%%writefile ./tf-provider/availability-domains.tf


# Source from https://registry.terraform.io/providers/oracle/oci/latest/docs/data-sources/identity_availability_domains

# Tenancy is the root or parent to all compartments.
# For this, use the value of <tenancy-ocid> for the compartment OCID.

data "oci_identity_availability_domains" "ads" {
  compartment_id = "<tenancy-ocid>"
}

In [ ]:
!code ./tf-provider/availability-domains.tf

In [ ]:
%%writefile ./tf-provider/outputs.tf


# Output the "list" of all availability domains.
output "all-availability-domains-in-your-tenancy" {
  value = data.oci_identity_availability_domains.ads.availability_domains
}

In [ ]:
!code ./tf-provider/outputs.tf

## Run Scripts

In [ ]:
!pwd

In [ ]:
!(cd tf-provider && tofu init) 

In [ ]:
!(cd tf-provider && ls -al) 

In [ ]:
!(cd tf-provider && tree .) 

In [ ]:
!(cd tf-provider && tofu validate)

In [ ]:
!(cd tf-provider && tofu plan -input=false)

In [ ]:
!(cd tf-provider && tofu apply -input=false -auto-approve)

In [ ]:
!(cd tf-provider && tofu output)

# Create a Compartment

[Create a Compartment](https://docs.oracle.com/en-us/iaas/developer-tutorials/tutorials/tf-compartment/01-summary.htm)

## Create Scripts

In [ ]:
!mkdir ./tf-compartment

In [ ]:
!cp ./tf-provider/provider.tf ./tf-compartment/provider.tf 

In [ ]:
%%writefile ./tf-compartment/compartment.tf


resource "oci_identity_compartment" "tf-compartment" {
    # Required
    compartment_id = "<tenancy-ocid>"
    description = "<your-description>"
    name = "<your-compartment-name>"
}

Replace your-compartment-name with a name of your choice.

In [ ]:
!code ./tf-compartment/compartment.tf

In [ ]:
%%writefile ./tf-compartment/outputs.tf


# Outputs for compartment
output "compartment-name" {
  value = oci_identity_compartment.tf-compartment.name
}

output "compartment-OCID" {
  value = oci_identity_compartment.tf-compartment.id
}

In [ ]:
!code ./tf-compartment/outputs.tf

## Run Scripts

In [ ]:
!pwd

In [ ]:
!(cd tf-compartment && ls -al) 

In [ ]:
!(cd tf-compartment && tree .) 

In [ ]:
!(cd tf-compartment && tofu init) 

In [ ]:
!(cd tf-compartment && tofu validate)

In [ ]:
!(cd tf-compartment && tofu plan -input=false)

In [ ]:
!(cd tf-compartment && tofu apply -input=false -auto-approve)

In [ ]:
!(cd tf-compartment && tofu output)

# Create a Virtual Cloud Network

[Create a Virtual Cloud Network](https://docs.oracle.com/en-us/iaas/developer-tutorials/tutorials/tf-vcn/01-summary.htm)

[VCN (basics)](https://isaac-exe.gitbook.io/various-tutorials/tutorials/untitled/vcn-basics)

## Create a Basic Network

In [ ]:
!mkdir ./tf-vcn

In [ ]:
!cp ./tf-provider/provider.tf ./tf-vcn/provider.tf 

In [ ]:
%%writefile ./tf-vcn/vcn.tf


# https://github.com/oracle-terraform-modules/terraform-oci-vcn

resource "oci_core_vcn" "vcn" {
  # We still allow module users to declare a cidr using `vcn_cidr` instead of the now recommended `vcn_cidrs`, but internally we map both to `cidr_blocks`
  # The module always use the new list of string structure and let the customer update his module definition block at his own pace.
  cidr_blocks    = ["10.0.0.0/16"]
  compartment_id = var.compartment_id
  display_name   = "vcn"
  dns_label      = "vcn"
  is_ipv6enabled = false

  freeform_tags = {
    terraformed = "Please do not edit manually"
    module      = "oracle-terraform-modules/vcn/oci"
  }
    
  defined_tags  = null

  lifecycle {
    ignore_changes = [defined_tags, dns_label, freeform_tags]
  }
}

## Customise the Network

### Create a Security List for the Public Subnet

In [ ]:
%%writefile ./tf-vcn/public-security-list.tf


# Source from https://registry.terraform.io/providers/oracle/oci/latest/docs/resources/core_security_list

resource "oci_core_security_list" "public-security-list"{

# Required
  compartment_id = var.compartment_id
  vcn_id = oci_core_vcn.vcn.id

# Optional
  display_name = "security-list-for-public-subnet"
    
  
  egress_security_rules {
    stateless = false
    destination = "0.0.0.0/0"
    destination_type = "CIDR_BLOCK"
    protocol = "all" 
  }

 
  ingress_security_rules {
    stateless   = false
    source      = "0.0.0.0/0"
    source_type = "CIDR_BLOCK"

    # Allow SSH (TCP port 22)
    protocol = "6"  # TCP
    tcp_options {
      min = 22
      max = 22
    }
  }
    

  ingress_security_rules { 
    stateless   = false
    source      = "10.1.0.0/16"
    source_type = "CIDR_BLOCK"
    # Get protocol numbers from https://www.iana.org/assignments/protocol-numbers/protocol-numbers.xhtml ICMP is 1  
    protocol    = "1"

    # For ICMP type and code see: https://www.iana.org/assignments/icmp-parameters/icmp-parameters.xhtml
    icmp_options {
      type = 8
      code = 0  
    } 
  }


}

### Create a Public Subnet

In [ ]:
%%writefile ./tf-vcn/public-subnet.tf


# Source from https://registry.terraform.io/providers/oracle/oci/latest/docs/resources/core_subnet
# https://docs.oracle.com/en-us/iaas/tools/terraform-provider-oci/5.24/docs/r/core_subnet.html

resource "oci_core_subnet" "vcn-public-subnet"{

  # Required
  compartment_id = var.compartment_id
  vcn_id         = oci_core_vcn.vcn.id
  cidr_block     = "10.0.0.0/24"
 
  # Optional
  route_table_id      = oci_core_route_table.public-route-table.id
  security_list_ids   = [oci_core_security_list.public-security-list.id]
  display_name        = "public-subnet"
  dns_label           = "public"
}

### Create Internet Gateway

In [ ]:
%%writefile ./tf-vcn/internet-gateway.tf


resource "oci_core_internet_gateway" "internet-gateway" {
  compartment_id = var.compartment_id
  display_name = "internet-gateway"
  vcn_id = oci_core_vcn.vcn.id
}

### Create Public Subnet Route Table

In [ ]:
%%writefile ./tf-vcn/public-route-table.tf


resource "oci_core_route_table" "public-route-table" {

  compartment_id = var.compartment_id
  vcn_id = oci_core_vcn.vcn.id
  display_name = "public-route-table"

  route_rules {
    destination_type = "CIDR_BLOCK"
    destination = "0.0.0.0/0"
    network_entity_id = oci_core_internet_gateway.internet-gateway.id
  }

  route_rules {
    destination_type = "CIDR_BLOCK"
    destination = "10.1.0.0/16"
    network_entity_id = oci_core_local_peering_gateway.local-peering-gateway.id
  }
    
}


### Create Local Peering Gateway

In [ ]:
%%writefile ./tf-vcn/local-peering-gateway.tf


resource "oci_core_local_peering_gateway" "local-peering-gateway" {
  compartment_id = var.compartment_id
  display_name   = "local-peering-gateway"
  vcn_id = oci_core_vcn.vcn.id
}

### Variables

In [ ]:
%%writefile ./tf-vcn/variables.tf


variable "compartment_id" {
  # This is the compartment ID of this VCN.
  description = "Compartment ID"    
  type        = string
  default     = "<compartment-ocid>"
}

In [ ]:
!code ./tf-vcn/variables.tf

### Outputs

In [ ]:
%%writefile ./tf-vcn/outputs.tf


# Outputs for public security list
output "public-security-list-name" {
  value = oci_core_security_list.public-security-list.display_name
}
output "public-security-list-OCID" {
  value = oci_core_security_list.public-security-list.id
}

# Outputs for public subnet
output "public-subnet-name" {
  value = oci_core_subnet.vcn-public-subnet.display_name
}
output "public-subnet-OCID" {
  value = oci_core_subnet.vcn-public-subnet.id
}

# Outputs for local peering gateway
output "local-peering-gateway-name" {
  value = oci_core_local_peering_gateway.local-peering-gateway.display_name
}
output "local-peering-gateway-OCID" {
  value = oci_core_local_peering_gateway.local-peering-gateway.id
}

In [ ]:
!code ./tf-vcn/outputs.tf

## Run Scripts

In [ ]:
!pwd

In [ ]:
!(cd tf-vcn && ls -al) 

In [ ]:
!(cd tf-vcn && tree .) 

In [ ]:
!(cd tf-vcn && tofu init) 

In [ ]:
!(cd tf-vcn && tofu validate)

In [ ]:
!(cd tf-vcn && tofu plan -input=false)

In [ ]:
!(cd tf-vcn && tofu apply -input=false -auto-approve)

In [ ]:
!(cd tf-vcn && tofu output)

# Create a Peer Compute Instance

[Create a Compute Instance](https://docs.oracle.com/en-us/iaas/developer-tutorials/tutorials/tf-compute/01-summary.htm)

## Create SSH Encryption Keys
Execute the following command in the terminal:
```bash
ssh-keygen -t rsa -b 2048 -C "" -f ~/.ssh/peer_1_vm_key
```
## Create Scripts

In [ ]:
!mkdir tf-peer_vm

In [ ]:
!cp ./tf-provider/provider.tf ./tf-peer_vm/peer_vm.tf 

In [ ]:
!cp ./tf-vcn/variables.tf ./tf-peer_vm/variables.tf 

In [ ]:
!cp ./tf-provider/availability-domains.tf ./tf-peer_vm/availability-domains.tf

In [ ]:
%%writefile ./tf-peer_vm/peer_vm.tf


resource "oci_core_instance" "peer-vm" {
  # Required
  availability_domain = data.oci_identity_availability_domains.ads.availability_domains[0].name
  compartment_id = var.compartment_id
  #shape = "VM.Standard.E2.1.Micro"  # AMD
  shape = "VM.Standard.A1.Flex"  # ARM

  # when using Flex you need a shape_config
  shape_config {

        #Optional
        #baseline_ocpu_utilization = var.instance_shape_config_baseline_ocpu_utilization
        memory_in_gbs = 1
        #nvmes = var.instance_shape_config_nvmes
        ocpus = 1
        #vcpus = var.instance_shape_config_vcpus
  }
    
  source_details {
    # https://docs.oracle.com/en-us/iaas/images/ 
    source_id = "<source-ocid>"
    source_type = "image"
  }

  # Optional
  display_name = "peer-vm"
  create_vnic_details {
    assign_public_ip = true
    # Public subnet
    # Find this in your VCN outputs.  
    subnet_id = "<subnet-ocid>"
  }
  metadata = {
    ssh_authorized_keys = file("<ssh-public-key-path>")
  } 
  preserve_boot_volume = false
}

In [16]:
!code ./tf-peer_vm/peer_vm.tf 

In [ ]:
%%writefile ./tf-peer_vm/outputs.tf


# The "name" of the availability domain to be used for the compute instance.
output "name-of-first-availability-domain" {
  value = data.oci_identity_availability_domains.ads.availability_domains[0].name
}

# Outputs for compute instance
output "public-ip-for-compute-instance" {
  value = oci_core_instance.peer-vm.public_ip
}

output "private-ip-for-compute-instance" {
  value = oci_core_instance.peer-vm.private_ip
}

output "instance-name" {
  value = oci_core_instance.peer-vm.display_name
}

output "instance-OCID" {
  value = oci_core_instance.peer-vm.id
}

output "instance-region" {
  value = oci_core_instance.peer-vm.region
}

output "instance-shape" {
  value = oci_core_instance.peer-vm.shape
}

output "instance-state" {
  value = oci_core_instance.peer-vm.state
}

output "instance-OCPUs" {
  value = oci_core_instance.peer-vm.shape_config[0].ocpus
}

output "instance-memory-in-GBs" {
  value = oci_core_instance.peer-vm.shape_config[0].memory_in_gbs
}

output "time-created" {
  value = oci_core_instance.peer-vm.time_created
}

In [ ]:
!code ./tf-peer_vm/outputs.tf 

## Run Scripts

In [10]:
!pwd

/home/harley/cloud/oci-notebook-vcn-peer1


In [ ]:
!(cd tf-peer_vm && ls -al) 

In [ ]:
!(cd tf-peer_vm && tree .) 

In [ ]:
!(cd tf-peer_vm && tofu init) 

In [ ]:
!(cd tf-peer_vm && tofu validate)

In [ ]:
!(cd tf-peer_vm && tofu plan -input=false)

In [ ]:
!(cd tf-peer_vm && tofu apply -input=false -auto-approve)

In [ ]:
!(cd tf-peer_vm && tofu output)

## Connect to the VM Instance
To connect to the VM:
```bash
ssh -i <ssh-private-key-path> ubuntu@<your-public-ip-address>
```

# Destroy Resources

In [ ]:
!(cd tf-peer_vm && tofu destroy -input=false -auto-approve)

In [ ]:
!(cd tf-vcn && tofu destroy -input=false -auto-approve)

In [ ]:
!(cd tf-compartment && tofu destroy -input=false -auto-approve)

In [ ]:
!(cd tf-provider && tofu destroy -input=false -auto-approve)